# Kafka Data Consumer

## I/ Objectifs du notebook

## II/ Mécanisme de collecte en streaming des données


## III/ Mécanismes de validation et corrections des données

## IV/ Mécanismes de mise en base des données